In [15]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers ,activations,models,preprocessing,utils
import bs4 
import urllib.request
import re
import nltk

In [3]:
link=urllib.request.urlopen('https://en.wikibooks.org/wiki/Data_Science:_An_Introduction')

In [4]:
li=link.read()

In [5]:
data=bs4.BeautifulSoup(li,'lxml')

In [6]:
data

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Data Science: An Introduction - Wikibooks, open books for an open world</title>
<script>document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )enwikibooksmwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"667a6864-c9d1-4218-b24f-fbd510b1c998","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Data_Science:_An_Introduction","wgTitle":"Data Science: An Introduction","wgCurRe

In [8]:
data_para=data.find_all('p')

In [9]:
data_text=''
for para in data_para:
    data_text+=para.text


In [10]:
data_text

'\n\nWikibook\n\n\nData Science: An Introduction\nEdit This Box\n\nThis book is a very basic introduction to data science.   It is designed for the advanced high school student or average college freshman with a high school-level understanding of math, science, word processing and spreadsheets.  No understanding of computer science is assumed.  The main emphasis of this book is to help students think about the world in data science terms.  While some elementary data science skills will be taught, the point is not skill development, but rather critical thinking and problem solving development.  These are skills that can be successfully applied to all phases of life, not just data science. \nData science--as a profession and as an academic discipline unto itself—is new, having been born in the first decade of the 21st century.  It is a child born of the mature parental disciplines of scientific methods, data and software engineering, statistics, and visualization.  This book is not inten

In [59]:
import os
os.environ["OPENAI_API_KEY"]="Your_api_key"

In [60]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma,Pinecone
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
import nltk

In [86]:
txt_splitter=CharacterTextSplitter(chunk_size=100,chunk_overlap=0)

In [87]:
texts=txt_splitter.split_text(data_text)

In [88]:
embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [89]:
docsearch = Chroma.from_texts(texts, embedding, metadatas=[{"source": i} for i in range(len(texts))])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [96]:
query = "science"
docs = docsearch.similarity_search(query)

In [97]:
print(docs[1])

page_content='This book is a very basic introduction to data science.   It is designed fothe advanced high school student oaverage college freshman with a high school-level understanding of math, science, word processing and spreadsheets.  No understanding of computescience is assumed.  The main emphasis of this book is to help students think about the world in data science terms.  While some elementary data science skills will be taught, the point is not skill development, but rathecritical thinking and problem solving development.  These are skills that can be successfully applied to all phases of life, not just data science. \nData science--as a profession and as an academic discipline unto itself—is new, having been born in the first decade of the 21st century.  It is a child born of the mature parental disciplines of scientific methods, data and software engineering, statistics, and visualization.  This book is not intended to do justice to any of those disciplines by themselves, 

In [98]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory

In [99]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=prompt)

In [103]:
query = " data science"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Data science is a field of study that combines scientific methods, data engineering, statistics, and visualization to analyze and interpret data. It is used to gain insights and make predictions from data. Data science is also used to develop new products and services, and to improve existing ones.'}

In [104]:
print(chain.memory.buffer)


Human: What did the president say about Justice Breyer
AI:  The President has not made any public statements about Justice Breyer.
Human: What is the meaning of data science
AI:  Data science is a field of study that combines scientific methods, data engineering, statistics, and visualization to analyze and interpret data. It is used to gain insights and make predictions from data. Data science is also used to develop new products and services, and to improve existing ones.
Human:  data science
AI:  Data science is a field of study that combines scientific methods, data engineering, statistics, and visualization to analyze and interpret data. It is used to gain insights and make predictions from data. Data science is also used to develop new products and services, and to improve existing ones.


In [70]:
output = chatgpt_chain.predict(human_input="who are you")



> Entering new LLMChain chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant 

In [11]:
data_text=re.sub('r\[[0-9]*\]',' ',data_text)
data_text=re.sub('r\s+','',data_text)

In [12]:
data_text

'\n\nWikibook\n\n\nData Science: An Introduction\nEdit This Box\n\nThis book is a very basic introduction to data science.   It is designed fothe advanced high school student oaverage college freshman with a high school-level understanding of math, science, word processing and spreadsheets.  No understanding of computescience is assumed.  The main emphasis of this book is to help students think about the world in data science terms.  While some elementary data science skills will be taught, the point is not skill development, but rathecritical thinking and problem solving development.  These are skills that can be successfully applied to all phases of life, not just data science. \nData science--as a profession and as an academic discipline unto itself—is new, having been born in the first decade of the 21st century.  It is a child born of the mature parental disciplines of scientific methods, data and software engineering, statistics, and visualization.  This book is not intended to d

In [16]:
sem=nltk.sent_tokenize(data_text)
word=nltk.word_tokenize(data_text)

In [17]:
word

['Wikibook',
 'Data',
 'Science',
 ':',
 'An',
 'Introduction',
 'Edit',
 'This',
 'Box',
 'This',
 'book',
 'is',
 'a',
 'very',
 'basic',
 'introduction',
 'to',
 'data',
 'science',
 '.',
 'It',
 'is',
 'designed',
 'fothe',
 'advanced',
 'high',
 'school',
 'student',
 'oaverage',
 'college',
 'freshman',
 'with',
 'a',
 'high',
 'school-level',
 'understanding',
 'of',
 'math',
 ',',
 'science',
 ',',
 'word',
 'processing',
 'and',
 'spreadsheets',
 '.',
 'No',
 'understanding',
 'of',
 'computescience',
 'is',
 'assumed',
 '.',
 'The',
 'main',
 'emphasis',
 'of',
 'this',
 'book',
 'is',
 'to',
 'help',
 'students',
 'think',
 'about',
 'the',
 'world',
 'in',
 'data',
 'science',
 'terms',
 '.',
 'While',
 'some',
 'elementary',
 'data',
 'science',
 'skills',
 'will',
 'be',
 'taught',
 ',',
 'the',
 'point',
 'is',
 'not',
 'skill',
 'development',
 ',',
 'but',
 'rathecritical',
 'thinking',
 'and',
 'problem',
 'solving',
 'development',
 '.',
 'These',
 'are',
 'skills',


In [18]:
chars = sorted(list(set(word)))
chars_to_num=dict((c,i)for i,c in enumerate(chars))

In [19]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string



In [20]:
input_len = len(word)
vocab_len = len(chars)

In [21]:
input_len

1201

In [22]:
vocab_len

485

In [23]:
seq_length = 100
x= []
y= []

In [25]:
for i in range(0, input_len - seq_length, 1):
    in_seq = word[i:i + seq_length]
    out_seq = word[i + seq_length]
    x.append([chars_to_num[char] for char in in_seq])
    y.append(chars_to_num[out_seq])
    
n_patterns = len(x)

In [34]:
y.shape

(1101, 485)

In [26]:
n_patterns

1102

In [27]:
import numpy
X = numpy.reshape(x, (n_patterns, seq_length, 1))
X = X/float(vocab_len)
y = tf.keras.utils.to_categorical(y)

In [28]:
from keras.layers import Input, LSTM, Dense,Dropout
from keras.models import Model

model = tf.keras.Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

2023-03-07 14:58:59.316063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 14:58:59.316086: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 14:58:59.316108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2023-03-07 14:58:59.316380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [35]:
wnlem=nltk.stem.WordNetLemmatizer()

In [37]:
def perform_lemn(tokens):
    return[wnlem.lemmatize(token) for token in tokens]
    

In [39]:
import string

In [40]:
pr=dict((ord(punctuation),None) for punctuation in string.punctuation)

In [41]:
def get_processed_text(document):
    return perform_lemn( nltk.word_tokenize(document.lower().translate(pr)))

In [42]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

In [56]:
def response(user_response):
    robo_response=" "
    sem.append(user_response)
    Tfidvec=TfidfVectorizer(tokenizer=get_processed_text,stop_words='english')
    tfid=Tfidvec.fit_transform(sem)
    vals=cosine_similarity(tfid[-1],tfid)
    idx=vals.argsort()[0][-2]
    flat=vals.flatten()
    flat.sort()
    req_tfid=flat[-2]
    if(req_tfid==0):
        robo_response=robo_response+"I am sorry I didn't understand"
        return robo_response
    else:
        robo_response=robo_response+sem[idx]
        return robo_response

In [57]:
flag=True
print("Bot is running")
while (flag==True):
    user_respone=input("YOU : ")
    user_respone=user_respone.lower()
    if user_respone!='bye':
        if(user_respone=='thanks' or user_respone=="thank you"):
            flag=False
            print("BOT : Welcome")
        else:
            if(greeting(user_respone)!=None):
                print("BOT :"+ greeting(user_respone))
            else:
                print("BOT :",end="")
                print(response(user_respone))
                sem.remove(user_respone)
    else:
        flag=False
        print("BOT : Bye take care")

Bot is running
YOU : Data
BOT : data
YOU : data scien
BOT : data
YOU : what is data science
BOT : data
YOU : Matlab
BOT : We know that intermediate oadvanced students would use othetools such as MySQL, PHP, Python, Java, Hadoop, HBase, AllegroGraph, Mahout, MATLAB, SPSS, SAS, etc.
YOU : Mysql
BOT : We know that intermediate oadvanced students would use othetools such as MySQL, PHP, Python, Java, Hadoop, HBase, AllegroGraph, Mahout, MATLAB, SPSS, SAS, etc.
YOU : hello world
BOT :hey
YOU : hi
BOT :*nods*
YOU : okk
BOT : I am sorry I didn't understand
YOU : home work
BOT : Most data scientists do not get to choose who they work with, but must learn to work with whomeveis assigned to theiteam.
YOU : end
BOT : We image that there will be more lecture periods toward the beginning of the semesteand more presentation and review days toward the end of the semester.
YOU : bye
BOT : Bye take care
